In [145]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest, mutual_info_regression

In [ ]:
df = pd.read_csv('../../../data/processed/mockup_dataset_road_v2.csv')

In [147]:
df.head()

,price,land_area,latitude,longitude,price_per_m2,geometry,index_right,population,h3,ADM3_EN,...,f_road,f_secondary,f_service,f_steps,f_tertiary,f_track,f_trunk,f_trunk_link,f_unclassified,f_unused
0,420000.0,420.0,11.5445,104.913586,1000.0,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Tuol Tumpung Ti Muoy,...,0,1,1,1,1,1,1,1,1,0
1,420000.0,420.0,11.5445,104.913586,1000.0,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Tuol Tumpung Ti Pir,...,0,1,1,1,1,1,1,1,1,0
2,420000.0,420.0,11.5445,104.913586,1000.0,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Boeng Trabaek,...,0,1,1,1,1,1,1,1,1,0
3,420000.0,420.0,11.5445,104.913586,1000.0,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Boeng Keng Kang Ti Bei,...,0,1,1,1,1,1,1,1,1,0
4,420000.0,420.0,11.5445,104.913586,1000.0,POINT (104.913586 11.5445),53909.0,23239.0,8865846ac7fffff,Tuol Svay Prey Ti Muoy,...,0,1,1,1,1,1,1,1,1,0


In [148]:
# df = df[df['price_per_m2'] >= 100].copy()

In [149]:
# df[['bedrooms', 'bathrooms']] = df[['bedrooms', 'bathrooms']].fillna(0)

In [150]:
# # Calculate mean, max, min of 'price_per_m2' grouped by 'h_id'
# price_stats = df.groupby('address_line_2')['price_per_m2'].agg(['mean','median', 'max', 'min']).reset_index()

# # Merge these stats back to the original dataframe on 'h_id'
# df = df.merge(price_stats, on='address_line_2', how='left', suffixes=('', '_price_per_m2'))

# # The columns 'mean', 'max', 'min' are now added to df

In [151]:
# df.drop(columns=['price_display'], inplace=True)

In [152]:
grouped = df.groupby('h3')['price_per_m2']
df['h3_price_mean'] = df['h3'].map(grouped.mean())
df['h3_price_max'] = df['h3'].map(grouped.max())
df['h3_price_median'] = df['h3'].map(grouped.median())
df['h3_price_min'] = df['h3'].map(grouped.min())

In [153]:
df.drop(columns=['price', 'geometry', 'polygon_geom', 'land_area'], inplace=True)

In [154]:
df.dropna(inplace=True)

In [155]:
y = df[['price_per_m2']]
X = df.drop(columns=['price_per_m2'])

In [156]:
X.describe()

,latitude,longitude,index_right,population,near_Koh_Pich_in_km,Koh_Pich_nearest,Koh_Pich_1_2km,Koh_Pich_2_3km,Koh_Pich_3_5km,Koh_Pich_5_10km,...,f_tertiary,f_track,f_trunk,f_trunk_link,f_unclassified,f_unused,h3_price_mean,h3_price_max,h3_price_median,h3_price_min
count,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,10278.000000,...,10278.0,10278.0,10278.000000,10278.000000,10278.0,10278.0,10278.000000,10278.000000,10278.000000,10278.000000
mean,11.555691,104.914746,54024.236038,16188.920607,3.670169,0.005254,0.086106,0.227476,0.431699,0.204125,...,1.0,1.0,0.999903,0.999903,1.0,0.0,2406.336900,6095.679475,2142.092317,382.429203
std,0.026172,0.022643,730.468752,8663.614551,2.481848,0.072297,0.280535,0.419223,0.495337,0.403081,...,0.0,0.0,0.009864,0.009864,0.0,0.0,489.146213,1687.049389,478.061870,430.756612
min,11.430122,104.748841,38893.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,0.000000,0.000000,1.0,0.0,100.000000,100.000000,100.000000,100.000000
25%,11.544500,104.903494,53909.000000,7658.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.000000,1.000000,1.0,0.0,2184.343780,5537.037037,1829.988194,110.000000
50%,11.550000,104.920096,53920.000000,16252.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.000000,1.000000,1.0,0.0,2423.357695,6363.636364,2196.712018,216.382979
75%,11.571708,104.925100,53953.000000,23239.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,1.0,1.0,1.000000,1.000000,1.0,0.0,2748.106096,6465.517241,2348.279152,457.327586
max,11.720574,104.996857,71495.000000,39519.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.0,1.0,1.000000,1.000000,1.0,0.0,6435.643564,8870.967742,6435.643564,6435.643564


In [157]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train.shape, X_test.shape

In [158]:
cat_cols = X.select_dtypes(include=['object', 'category']).columns


In [159]:
cat_cols

Index(['h3', 'ADM3_EN', 'ADM2_EN', 'ADM1_EN'], dtype='object')

In [160]:
# Temporarily encode categorical columns for feature selection
X_temp = X.copy()
if len(cat_cols) > 0:
    encoder = OrdinalEncoder()
    X_temp[cat_cols] = encoder.fit_transform(X_temp[cat_cols])


In [ ]:
# Feature selection 
selector = SelectKBest(mutual_info_regression, k=10)
selector.fit(X_temp, y)
selected_features = X.columns[selector.get_support()]


d:\anaconda\envs\env_v3.10\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [162]:
# Subset original data to selected features
X_selected = X[selected_features]

In [163]:
X_selected

,latitude,longitude,n_cafe_5km,n_cafe_in_1km_to_2km,n_cafe_in_3km_to_5km,n_gas_station_5km,n_hospital_in_1km_to_2km,n_hospital_in_2km_to_3km,n_hospital_in_3km_to_5km,n_hotel_in_1km_to_2km,...,n_resturant_in_3km_to_5km,n_super_market_5km,n_super_market_in_3km_to_5km,n_atm_in_1km_to_2km,n_atm_in_2km_to_3km,n_atm_in_3km_to_5km,h3_price_mean,h3_price_max,h3_price_median,h3_price_min
0,11.544500,104.913586,177,36,85,145,36,49,22,5,...,68,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.000000
1,11.544500,104.913586,177,36,85,145,36,49,22,5,...,68,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.000000
2,11.544500,104.913586,177,36,85,145,36,49,22,5,...,68,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.000000
3,11.544500,104.913586,177,36,85,145,36,49,22,5,...,68,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.000000
4,11.544500,104.913586,177,36,85,145,36,49,22,5,...,68,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,11.575837,104.920096,175,31,77,143,49,23,21,36,...,8,98,30,77,27,31,2606.024445,8870.967742,2115.427303,104.166667
10288,11.575837,104.920096,175,31,77,143,49,23,21,36,...,8,98,30,77,27,31,2606.024445,8870.967742,2115.427303,104.166667
10289,11.575837,104.920096,175,31,77,143,49,23,21,36,...,8,98,30,77,27,31,2606.024445,8870.967742,2115.427303,104.166667
10290,11.439340,104.849064,1,0,1,3,0,0,0,0,...,0,0,0,0,0,0,100.000000,100.000000,100.000000,100.000000


In [164]:
# One-hot encode only the selected categorical features
selected_cat_cols = [col for col in selected_features if col in cat_cols]
X_final = pd.get_dummies(X_selected, columns=selected_cat_cols, drop_first=True)

# Only convert one-hot columns to int
one_hot_cols = [col for col in X_final.columns if any(cat in col for cat in selected_cat_cols)]
X_final[one_hot_cols] = X_final[one_hot_cols].astype(int)

In [165]:
X_final

,latitude,longitude,n_cafe_5km,n_cafe_in_1km_to_2km,n_cafe_in_3km_to_5km,n_gas_station_5km,n_hospital_in_1km_to_2km,n_hospital_in_2km_to_3km,n_hospital_in_3km_to_5km,n_hotel_in_1km_to_2km,...,n_resturant_in_3km_to_5km,n_super_market_5km,n_super_market_in_3km_to_5km,n_atm_in_1km_to_2km,n_atm_in_2km_to_3km,n_atm_in_3km_to_5km,h3_price_mean,h3_price_max,h3_price_median,h3_price_min
0,11.544500,104.913586,177,36,85,145,36,49,22,5,...,68,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.000000
1,11.544500,104.913586,177,36,85,145,36,49,22,5,...,68,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.000000
2,11.544500,104.913586,177,36,85,145,36,49,22,5,...,68,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.000000
3,11.544500,104.913586,177,36,85,145,36,49,22,5,...,68,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.000000
4,11.544500,104.913586,177,36,85,145,36,49,22,5,...,68,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10287,11.575837,104.920096,175,31,77,143,49,23,21,36,...,8,98,30,77,27,31,2606.024445,8870.967742,2115.427303,104.166667
10288,11.575837,104.920096,175,31,77,143,49,23,21,36,...,8,98,30,77,27,31,2606.024445,8870.967742,2115.427303,104.166667
10289,11.575837,104.920096,175,31,77,143,49,23,21,36,...,8,98,30,77,27,31,2606.024445,8870.967742,2115.427303,104.166667
10290,11.439340,104.849064,1,0,1,3,0,0,0,0,...,0,0,0,0,0,0,100.000000,100.000000,100.000000,100.000000


In [166]:
df_KBest = pd.concat([X_final, y], axis=1)


In [167]:
df_KBest.head()

,latitude,longitude,n_cafe_5km,n_cafe_in_1km_to_2km,n_cafe_in_3km_to_5km,n_gas_station_5km,n_hospital_in_1km_to_2km,n_hospital_in_2km_to_3km,n_hospital_in_3km_to_5km,n_hotel_in_1km_to_2km,...,n_super_market_5km,n_super_market_in_3km_to_5km,n_atm_in_1km_to_2km,n_atm_in_2km_to_3km,n_atm_in_3km_to_5km,h3_price_mean,h3_price_max,h3_price_median,h3_price_min,price_per_m2
0,11.5445,104.913586,177,36,85,145,36,49,22,5,...,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.0,1000.0
1,11.5445,104.913586,177,36,85,145,36,49,22,5,...,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.0,1000.0
2,11.5445,104.913586,177,36,85,145,36,49,22,5,...,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.0,1000.0
3,11.5445,104.913586,177,36,85,145,36,49,22,5,...,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.0,1000.0
4,11.5445,104.913586,177,36,85,145,36,49,22,5,...,95,35,28,59,83,2748.106096,6465.517241,2348.279152,110.0,1000.0


In [ ]:
df_KBest.to_csv('../../../data/preprocessed/land_dataset_feature_selection_Kbest_v1.csv', index=False)